In [6]:
import os
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from utils.transforms import HairRemoval, Composer
from utils.utils import export_experiment
from descriptors.shape import HOGDescriptor
from utils.segmentation import ThresholdingSegmentation
from descriptors.stats import IntensityStatsGridDescriptor
from descriptors.texture import LBPDescriptor, GLCMDescriptor, GaborFilterDescriptor
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor

In [24]:
## Classes
CLASSES = ['mel', 'bcc', 'scc']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge2')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()
exp_name = 'multi-class_classification'

In [25]:
transforms_composer = Composer([
                                HairRemoval(),
                                ])

## Featrure Extraction

### Descriptors

In [26]:
## Define parameters
params = {
    'color_layout': {
        'grid_x': 1,
        'grid_y': 1,
    },
    'intensity_stats': {
        'grid_x': 1,
        'grid_y': 1,
    },
    'color': {
        'bins': (8, 12, 3),
        'grid_x': 1,
        'grid_y': 1,
    },
    'glcm': {
        'distances': [1],
        'angles': [0, np.pi/4, np.pi/2, 3*np.pi/4],
        'levels': 8,
        'grid_x': 1,
        'grid_y': 1,
    },
    'lbp': {
        'radius': 3,
        'n_points': 16,
        'grid_x': 1,
        'grid_y': 1,
    },
}

In [27]:
modes = ['train', 'val']

## Descriptors
color_layout_descriptor = ColorLayoutDescriptor(**params['color_layout'])
intensity_stats_grid_descriptor = IntensityStatsGridDescriptor(**params['intensity_stats'])
color_descriptor = ColorDescriptor(**params['color'])
glcm_descriptor = GLCMDescriptor(**params['glcm'])
lbp_descriptor = LBPDescriptor(**params['lbp'])
# lbp2_descriptor = LBPDescriptor(**params['lbp2'])
# lbp3_descriptor = LBPDescriptor(**params['lbp3'])
# color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8, grid_x=3, grid_y=3)
# gabor_filter_descriptor = GaborFilterDescriptor(frequencies=[0.1, 0.2, 0.3], orientations=[0, np.pi/4, np.pi/2, 3*np.pi/4])

features_dict = {}

for mode in modes:

    ## Data loader
    ### Limit the number of samples to 200 for training and load all samples for validation
    max_samples = None if mode == 'train' else None
    ### Balance the dataset for training
    balance = None if mode == 'train' else False
    dataloader = DataLoader(data_folder, mode, 
                            shuffle=True, 
                            ignore_folders=['black_background', '.DS_Store'], 
                            max_samples=max_samples, 
                            balance=balance,
                            transforms=None, 
                            classes=CLASSES, 
                            mask=False)

    ## Extract features
    features = []
    labels = []
    for i, (img, label, mask, path) in tqdm(enumerate(dataloader), total=len(dataloader), desc=f'Extracting features for {mode}'):
        color_features = color_descriptor.extract(img, mask=None)
        # color_layout_features = color_layout_descriptor.extract(img, mask=None)
        intensity_stats_grid_features = intensity_stats_grid_descriptor.extract(img, mask=None)
        glcm_features, glcm_img = glcm_descriptor.extract(img, mask=None)
        lbp_features, lbp_img = lbp_descriptor.extract(img, mask=None)
        # lbp2_features, lbp2_img = lbp2_descriptor.extract(img, mask=None)
        # lbp3_features, lbp3_img = lbp3_descriptor.extract(img, mask=None)
        # color_cooccurrence_matrix_features = color_cooccurrence_matrix_descriptor.extract(img, mask=None)
        # gabors_features = gabor_filter_descriptor.extract(img, mask=None)
        features.append(np.concatenate([lbp_features, glcm_features, color_features, intensity_stats_grid_features], axis=0))        
        
        ## add label
        labels.append(label)
        
    
    ## Save features to disk
    features = np.array(features)
    labels = np.array(labels)
    features_with_labels = np.concatenate([features, labels.reshape(-1, 1)], axis=1)
    features_dict[mode] = features_with_labels

Extracting features for val: 100%|██████████| 1270/1270 [02:50<00:00,  7.44it/s]


## Training

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [29]:
mode = 'train'
# model = SVC(kernel='rbf', C=5.0, random_state=42, degree=5)
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, n_estimators=3000, learning_rate=0.2, random_state=42)

In [30]:
features = features_dict[mode][:, :-1]
labels = features_dict[mode][:, -1]

X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)

In [31]:
scaler = StandardScaler()
model = Pipeline([('scaler', scaler), ('model', xgb_clf)])

In [32]:
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.2,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=3000, n_jobs=None, num_class=3,
                               num_parallel_tree=None, ...))])

## Validation

In [20]:
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred, target_names=CLASSES))
print("Kappa score: ", cohen_kappa_score(y_val, y_pred))

              precision    recall  f1-score   support

         mel       0.93      0.93      0.93      1085
         bcc       0.90      0.89      0.89       797
         scc       0.91      0.94      0.92       602

    accuracy                           0.92      2484
   macro avg       0.91      0.92      0.92      2484
weighted avg       0.92      0.92      0.92      2484

Kappa score:  0.8714400475847942


## Testing

In [21]:
mode = 'val'
features_test = features_dict[mode][:, :-1]
labels_test = features_dict[mode][:, -1]

y_pred = model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))
print("Kappa score: ", cohen_kappa_score(labels_test, y_pred))

              precision    recall  f1-score   support

         mel       0.91      0.91      0.91       678
         bcc       0.86      0.89      0.88       498
         scc       0.71      0.56      0.63        94

    accuracy                           0.88      1270
   macro avg       0.82      0.79      0.80      1270
weighted avg       0.87      0.88      0.87      1270

Kappa score:  0.7741637779678567


## Cross Validation

In [14]:
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold

# Initialize XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', objective='multi:softmax', num_class=3, n_estimators=500, learning_rate=0.1, n_jobs=-1)

# Define cross-validation strategy (e.g., 5-fold stratified)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and get accuracy scores for each fold
features_train = features_dict['train'][:, :-1]
labels_train = features_dict['train'][:, -1]
cv_scores = cross_val_score(xgb_clf, features_train, labels_train, cv=cv, scoring='accuracy')

# Output the results
print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())
print("Standard Deviation of CV Accuracy:", cv_scores.std())

Cross-Validation Accuracy Scores: [0.5 0.4 0.5 0.4 0.5]
Mean CV Accuracy: 0.45999999999999996
Standard Deviation of CV Accuracy: 0.04898979485566355


In [15]:
# Cross-validation predictions (optional)
features_test = features_dict['val'][:, :-1]
labels_test = features_dict['val'][:, -1]
cv_predictions = cross_val_predict(xgb_clf, features_test, labels_test, cv=cv)
print("Classification Report for CV Predictions:\n", classification_report(labels_test, cv_predictions))

Classification Report for CV Predictions:
               precision    recall  f1-score   support

         0.0       0.70      0.76      0.73        25
         1.0       0.74      0.68      0.71        25

    accuracy                           0.72        50
   macro avg       0.72      0.72      0.72        50
weighted avg       0.72      0.72      0.72        50



## Grid Search

In [22]:
from sklearn.model_selection import GridSearchCV

# Initialize the base model (XGBClassifier)
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', num_class=3, n_jobs=-1, random_state=42)
scaler_cv = StandardScaler()

model = Pipeline([('scaler', scaler_cv), ('model', xgb_clf)])

# Define the parameter grid for Grid Search
param_grid = {
    'model__n_estimators': [1500, 2000],  # Number of trees
    'model__learning_rate': [0.1, 0.2],  # Step size shrinkage
    # 'reg_lambda': [1.0, 0.8],  # L2 regularization term on weights
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                        scoring='accuracy',  # Use 'accuracy' as the evaluation metric
                        cv=5,  # 5-fold cross-validation
                        verbose=4, 
                        n_jobs=-1)  # Parallel processing

features_train = features_dict['train'][:, :-1]
labels_train = features_dict['train'][:, -1]
grid_search.fit(features_train, labels_train)

# Get the best parameters and best score from the grid search
print("Best Parameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END model__learning_rate=0.1, model__n_estimators=1500;, score=0.911 total time= 5.5min
[CV 5/5] END model__learning_rate=0.1, model__n_estimators=1500;, score=0.919 total time= 5.5min
[CV 2/5] END model__learning_rate=0.1, model__n_estimators=1500;, score=0.912 total time= 5.5min
[CV 4/5] END model__learning_rate=0.1, model__n_estimators=1500;, score=0.911 total time= 5.5min
[CV 3/5] END model__learning_rate=0.1, model__n_estimators=1500;, score=0.921 total time= 5.5min
[CV 1/5] END model__learning_rate=0.1, model__n_estimators=2000;, score=0.911 total time= 6.8min
[CV 2/5] END model__learning_rate=0.1, model__n_estimators=2000;, score=0.914 total time= 6.8min
[CV 3/5] END model__learning_rate=0.1, model__n_estimators=2000;, score=0.920 total time= 6.9min
[CV 1/5] END model__learning_rate=0.2, model__n_estimators=1500;, score=0.907 total time= 4.4min
[CV 3/5] END model__learning_rate=0.2, model__n_estimators=1500;, s

In [23]:
best_model = grid_search.best_estimator_
features_test = features_dict['val'][:, :-1]
labels_test = features_dict['val'][:, -1]

y_pred = best_model.predict(features_test)
print(classification_report(labels_test, y_pred, target_names=CLASSES))
print("Kappa score: ", cohen_kappa_score(labels_test, y_pred))

              precision    recall  f1-score   support

         mel       0.91      0.92      0.92       678
         bcc       0.87      0.90      0.89       498
         scc       0.72      0.57      0.64        94

    accuracy                           0.89      1270
   macro avg       0.84      0.80      0.81      1270
weighted avg       0.88      0.89      0.88      1270

Kappa score:  0.7926612322104125


## Exporting Experiment

In [33]:
## Export experiment
notebook_name = 'MultiClass_aug.ipynb'
export_experiment(name=exp_name, params=params, feature_dict=features_dict , model=best_model, notebook_name=notebook_name)

<IPython.core.display.Javascript object>

Experiment 'multi-class_classification' saved at experiments/multi-class_classification_20241104_021757


## Feature Selection

In [20]:
features_train = features_dict['train'][:, :-1]
labels_train = features_dict['train'][:, -1]
X_train, X_val, y_train, y_val = train_test_split(features_train, labels_train, test_size=0.1, random_state=42, stratify=labels)

In [21]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd


# Train random forest and get feature importances
model = RandomForestClassifier()
model.fit(X_train, y_train)
importances = model.feature_importances_

# Display feature importances
feature_importances = pd.Series(importances)
print(feature_importances.sort_values(ascending=False))

2786    0.010357
1379    0.010292
1457    0.009750
1454    0.009593
590     0.009396
          ...   
1019    0.000000
1020    0.000000
1021    0.000000
1022    0.000000
2834    0.000000
Length: 2835, dtype: float64


In [22]:
#get the most important features
top_features = feature_importances.sort_values(ascending=False).index[:400]

# Retrain the model using only the top features
model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, n_estimators=1000, learning_rate=0.2, n_jobs=-1)
model.fit(X_train[:, top_features], y_train)

# Evaluate the model
y_pred = model.predict(X_val[:, top_features])
print(classification_report(y_val, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       1.00      0.67      0.80         3
      others       0.67      1.00      0.80         2

    accuracy                           0.80         5
   macro avg       0.83      0.83      0.80         5
weighted avg       0.87      0.80      0.80         5



In [ ]:
# test the model
features_test = features_dict['val'][:, :-1]
labels_test = features_dict['val'][:, -1]

y_pred = model.predict(features_test[:, top_features])
print(classification_report(labels_test, y_pred, target_names=CLASSES))

              precision    recall  f1-score   support

       nevus       0.58      0.84      0.69        25
      others       0.71      0.40      0.51        25

    accuracy                           0.62        50
   macro avg       0.65      0.62      0.60        50
weighted avg       0.65      0.62      0.60        50



## Weak Learner Ensemble

In [ ]:
import numpy as np
from sklearn.utils import resample
from sklearn.base import clone

class ModelEnsemble:
    def __init__(self, models, subsample_size=0.8, random_state=None):
        """
        Initializes the Model Ensemble with a given set of models.

        Args:
            models (list): List of initialized model instances (e.g., [XGBClassifier(), RandomForestClassifier()]).
            subsample_size (float): Proportion of the dataset to use for each model (between 0 and 1).
            random_state (int, optional): Random seed for reproducibility. Defaults to None.
        """
        self.models = models
        self.n_models = len(models)
        self.subsample_size = subsample_size
        self.random_state = random_state
        self.trained_models = []

    def fit(self, X, y):
        """Train each model on a random subset of the data."""
        self.trained_models = []
        for i, model in enumerate(self.models):
            # Clone the model to ensure independence
            model_copy = clone(model)
            
            # Sample a random subset of the data for this model
            X_subset, y_subset = resample(X, y, n_samples=int(self.subsample_size * len(X)),
                                        random_state=self.random_state + i if self.random_state else None)
            
            # Train the model on the subset
            model_copy.fit(X_subset, y_subset)
            self.trained_models.append(model_copy)

    def predict(self, X):
        """Predict using all models and return the majority vote for each instance."""
        predictions = np.array([model.predict(X).reshape(-1) for model in self.trained_models])
        
        # Take the majority vote along the columns
        majority_vote = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions.astype(np.int64))
        return majority_vote

    def predict_proba(self, X):
        """Predict class probabilities by averaging probabilities from each model."""
        probas = np.array([model.predict_proba(X) for model in self.trained_models if hasattr(model, "predict_proba")])
        avg_probas = np.mean(probas, axis=0)
        return avg_probas


In [39]:
X = features_dict['train'][:, :-1]
y = features_dict['train'][:, -1]

X_test = features_dict['val'][:, :-1]
y_test = features_dict['val'][:, -1]

In [48]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Initialize different models
models = [
    XGBClassifier(learning_rate=0.2, random_state=42, n_estimators=1000),
    RandomForestClassifier(random_state=42, n_estimators=1000),
    CatBoostClassifier(learning_rate=0.2, random_state=42, verbose=0, n_estimators=1000),
    GradientBoostingClassifier(learning_rate=0.2, random_state=42, n_estimators=1000),
]

In [50]:
def predict(self, X):
    """Predict using all models and return the majority vote for each instance."""
    predictions = np.array([model.predict(X).reshape(-1) for model in self.trained_models])
    
    # Take the majority vote along the columns
    majority_vote = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=predictions.astype(np.int64))
    return majority_vote

ensemble.predict = types.MethodType(predict, ensemble)

In [51]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Initialize ensemble with XGBoost parameters
# ensemble = ModelEnsemble(models=models, subsample_size=0.8, random_state=42)

# # Train the ensemble
# ensemble.fit(X_train, y_train)

# Make predictions
y_pred = ensemble.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Ensemble Accuracy: {accuracy:.4f}")

# If probability predictions are needed
y_proba = ensemble.predict_proba(X_val)

Ensemble Accuracy: 0.8593


In [52]:
## Evaluate the ensemble on the test set
y_pred = ensemble.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Ensemble Accuracy on Test Set: {accuracy:.4f}")

print(classification_report(y_test, y_pred, target_names=CLASSES))
print("Kappa score: ", cohen_kappa_score(y_test, y_pred))

Ensemble Accuracy on Test Set: 0.8307
              precision    recall  f1-score   support

         mel       0.88      0.90      0.89       678
         bcc       0.80      0.85      0.82       498
         scc       0.48      0.26      0.33        94

    accuracy                           0.83      1270
   macro avg       0.72      0.67      0.68      1270
weighted avg       0.82      0.83      0.82      1270

Kappa score:  0.688263500399589
